In [1]:
from peft import prepare_model_for_kbit_training,LoraConfig,get_peft_model,TaskType
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer,default_data_collator,BitsAndBytesConfig
from datasets import load_dataset
import torch

[2025-06-13 12:49:39,764] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0613 12:49:41.126000 2192 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


[2025-06-13 12:49:41,616] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


### QLoRa配置

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['fp32'],
                             )

### 模型加载

In [3]:
model_name="Qwen/Qwen3-0.6B"
model=AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',torch_dtype="auto",quantization_config=q_config)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [4]:
kbit_model = prepare_model_for_kbit_training(model)

### LoRa配置

In [5]:
lora_config = LoraConfig(
    r=4,  # LoRA矩阵的秩
    lora_alpha=32,  # LoRA alpha参数
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # 要应用LoRA的模块
    lora_dropout=0.05,  # Dropout概率
    bias="none",  # 是否训练偏置
    task_type="CAUSAL_LM",  # 任务类型
)

In [6]:
qlora_model = get_peft_model(kbit_model, lora_config)

In [7]:
qlora_model.print_trainable_parameters()

trainable params: 1,146,880 || all params: 597,196,800 || trainable%: 0.1920


In [8]:
qlora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

### 加载模型

In [9]:
dataset = load_dataset('json', data_files={
    'train': '../数据集/data/train.json',
    'validation': '../数据集/data/eval.json'
})

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['conversations'],
        num_rows: 30
    })
})

### 数据预处理

In [11]:
def process_fun(example):
    question=[]
    answer=[]
    for i in example['conversations']:
        for j in i:
            if j['from']=='human':
                question.append(j['value'])
            elif j['from']=='gpt':
                answer.append(j['value'])
    return {'question':question,'answer':answer}

In [12]:
process_data=dataset.map(process_fun,batched=True,remove_columns=dataset['train'].column_names) # 提取问题和答案数据预处理

In [13]:
process_data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 606
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 52
    })
})

In [14]:
process_data['train'][0]

{'question': '请问室温超导技术的难点在哪里,人类在什么时候可以实现室温超导?',
 'answer': '室温超导技术的难点在于目前尚未发现或创造出能在室温下保持超导性的材料。常见的超导体需要在极低温（如液氦温度，约-269°C）下才能表现出零电阻和完全抗磁性的超导特性。尽管科学家们在高温超导材料研究方面取得了进展，目前已知的高温超导体在实现超导状态时所需的温度仍远远低于室温。\n\n实现室温超导的难点主要有以下几个方面：\n\n1. 材料问题：高温超导体多为陶瓷材料，这类材料往往脆弱难以制成长导线，并且许多超导材料在高温、高压条件下稳定性较差。\n\n2. 理论挑战：超导机制通常基于量子物理理论。对于传统的低温超导体，BCS理论（巴丁-库珀-施里弗理论）能够很好地描述其超导机制。但对于高温超导体，至今还没有一个普遍认可且完整的理论框架。\n\n3. 技术限制：即使找到合适的材料或者理论模型，通过现有的技术制备高质量的样品也是非常困难的。要保持超导状态需要的一系列条件如纯度、晶格结构的完整性等，在室温下实现和维持更加困难。\n\n4. 经济和实用性：即便科学家们能发现或制造出在室温下工作的超导材料，但其制造成本、可持续性和实际应用的有效性都是可能的挑战。\n\n至于人类能在什么时候实现室温超导，目前无法给出一个确切的时间表。室温超导是物理学中的一个激动人心但又极具挑战性的前沿议题，其研究进展依赖于新材料的发现、理论物理的突破以及实验技术的进步。可以说，室温超导的实现还处于科学探索的阶段，是未来技术发展具有重要潜力的领域之一。'}

In [15]:
def tokenizer_fun(examples):
    # 构建完整的指令格式（问：{问题}\n答：{答案}）
    instructions = []
    for q, a in zip(examples['question'], examples['answer']):
        instruction = f"问：{q}\n答：{a}"
        instructions.append(instruction)
    
    encoded = tokenizer(
        instructions,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    labels = encoded["input_ids"].clone()
    
    # 定位"答："的位置，标记需要预测的部分
    answer_start_token = tokenizer.encode("答：", add_special_tokens=False)[0]
    
    # 遍历批次中的每个样本
    for i in range(len(labels)):
        # 找到每个样本中"答："的第一个token位置
        answer_positions = (labels[i] == answer_start_token).nonzero(as_tuple=True)[0]
        if len(answer_positions) > 0:
            # 只取第一个"答："的位置
            first_answer_pos = answer_positions[0]
            # 将"答："之前的token标记为-100（忽略计算损失）
            labels[i, :first_answer_pos] = -100
    
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": labels
    }

In [16]:
tokenized_dataset = process_data.map(
    tokenizer_fun,
    batched=True,
    remove_columns=process_data['train'].column_names
)

Map:   0%|          | 0/606 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 606
})

### 训练超参数

In [18]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="./Qlora_train_qwen_0.6B_model",
    logging_steps=100,
    logging_dir='./runs',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    gradient_accumulation_steps=4,  # 如果GPU内存有限
)

### 训练模型

In [19]:
trainer=Trainer(
    model=qlora_model,
    args=training_args,
    eval_dataset=tokenized_dataset["validation"],
    train_dataset=tokenized_dataset["train"],
    data_collator=default_data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [20]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
D:\Anaconda\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,5.917908
2,No log,4.967466
3,No log,4.604531


D:\Anaconda\Lib\site-packages\peft\utils\other.py:1110: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-0.6B/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002642CD0D040>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 31dd60a0-2466-4a3b-8c90-48d276cc40a0)') - silently ignoring the lookup for the file config.json in Qwen/Qwen3-0.6B.
  warnings.warn(
D:\Anaconda\Lib\site-packages\peft\utils\save_and_load.py:236: UserWarning: Could not find a config file in Qwen/Qwen3-0.6B - will assume that the vocabulary was not modified.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not pass

TrainOutput(global_step=57, training_loss=5.8237561677631575, metrics={'train_runtime': 1479.0308, 'train_samples_per_second': 1.229, 'train_steps_per_second': 0.039, 'total_flos': 2466370138669056.0, 'train_loss': 5.8237561677631575, 'epoch': 3.0})